In [10]:
import pandas as pd

In [11]:
df_sini = pd.read_feather("siniestros.feather")

In [12]:
df_ids = pd.read_feather("poliza_x_cliente.feather")

Verificación de cruce

In [13]:
len(set(df_ids["NUM_SECU_POL"]).intersection(set(df_sini["NUM_SECU_POL"]))) / len(df_ids)

0.02880643884618783

In [14]:
len(set(df_sini["NUM_SECU_POL"]).intersection(set(df_ids["NUM_SECU_POL"]))) / len(df_sini)

0.11397639023777408

### Agregamos CIF_ID a los Siniestros

In [15]:
df_sini_id = pd.merge(df_sini, df_ids, on=['NUM_SECU_POL'], how='left')

In [16]:
df_sini_id.drop_duplicates(subset=["NUM_SECU_POL", "FEC_DENU_SINI"], keep='last', inplace=True)

### Agrupamos siniestros por periodo

In [17]:
data = {fecha:i + 1 for i,fecha in enumerate(sorted(df_sini_id["FEC_DENU_SINI"].unique()))}

In [18]:
data

{'2018-11': 1,
 '2018-12': 2,
 '2019-01': 3,
 '2019-02': 4,
 '2019-03': 5,
 '2019-04': 6,
 '2019-05': 7,
 '2019-06': 8,
 '2019-07': 9,
 '2019-08': 10,
 '2019-09': 11,
 '2019-10': 12,
 '2019-11': 13}

In [19]:
df_sini_id["periodo_denu_sini"] = df_sini_id["FEC_DENU_SINI"].map(data)

In [20]:
df_sini_id["periodo_liquidacion_sini"] = df_sini_id["FECHA_LIQUIDACION"].map(data)

In [21]:
df_sini_id["periodo_rechazo_sini"] = df_sini_id["FECHA_RECHAZO"].map(data)

In [22]:
to_pivot = df_sini_id[["CIF_ID","NUM_SECU_POL","periodo_denu_sini","periodo_liquidacion_sini","periodo_rechazo_sini"]]

### Pivots por CIF_ID

In [23]:
pd.set_option("display.max_columns", 500)

In [24]:
df_sini = to_pivot.pivot_table(index='CIF_ID',columns=['periodo_denu_sini'], values=['NUM_SECU_POL','periodo_liquidacion_sini','periodo_rechazo_sini'],aggfunc='count', fill_value=0)

In [25]:
df_sini.head()

NUM_SECU_POL                                      \
periodo_denu_sini           1  2  3  4  5  6  7  8  9  10 11 12 13   
CIF_ID                                                               
1482.0                       0  0  0  0  0  0  0  0  0  0  0  0  1   
1559.0                       0  0  0  0  0  1  0  0  0  0  0  1  0   
4655.0                       0  0  0  0  0  0  0  0  0  0  0  1  0   
5231.0                       0  0  0  0  0  0  1  0  0  0  0  1  0   
7870.0                       0  0  0  0  1  0  0  0  0  0  0  0  0   

                  periodo_liquidacion_sini                                   \
periodo_denu_sini                       1  2  3  4  5  6  7  8  9  10 11 12   
CIF_ID                                                                        
1482.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
1559.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
4655.0                                   0  0  0  0  0  0  0  0  0  0  0  0   
5231.0                                   0  0  0  0  0  0  0  0  0  0  0  1   
7870.0                                   0  0  0  0  0  0  0  0  0  0  0  0   

                     periodo_rechazo_sini                                      
periodo_denu_sini 13                   1  2  3  4  5  6  7  8  9  10 11 12 13  
CIF_ID                                                                         
1482.0             0                    0  0  0  0  0  0  0  0  0  0  0  0  0  
1559.0             0                    0  0  0  0  0  0  0  0  0  0  0  0  0  
4655.0             0                    0  0  0  0  0  0  0  0  0  0  0  0  0  
5231.0             0                    0  0  0  0  0  0  0  0  0  0  0  0  0  
7870.0             0                    0  0  0  0  0  0  0  0  0  0  0  0  0

Reagrupamos los DF

In [26]:
df_sini = pd.DataFrame(df_sini.to_records())

In [27]:
df_sini.head()

,CIF_ID,"('NUM_SECU_POL', 1)","('NUM_SECU_POL', 2)","('NUM_SECU_POL', 3)","('NUM_SECU_POL', 4)","('NUM_SECU_POL', 5)","('NUM_SECU_POL', 6)","('NUM_SECU_POL', 7)","('NUM_SECU_POL', 8)","('NUM_SECU_POL', 9)","('NUM_SECU_POL', 10)","('NUM_SECU_POL', 11)","('NUM_SECU_POL', 12)","('NUM_SECU_POL', 13)","('periodo_liquidacion_sini', 1)","('periodo_liquidacion_sini', 2)","('periodo_liquidacion_sini', 3)","('periodo_liquidacion_sini', 4)","('periodo_liquidacion_sini', 5)","('periodo_liquidacion_sini', 6)","('periodo_liquidacion_sini', 7)","('periodo_liquidacion_sini', 8)","('periodo_liquidacion_sini', 9)","('periodo_liquidacion_sini', 10)","('periodo_liquidacion_sini', 11)","('periodo_liquidacion_sini', 12)","('periodo_liquidacion_sini', 13)","('periodo_rechazo_sini', 1)","('periodo_rechazo_sini', 2)","('periodo_rechazo_sini', 3)","('periodo_rechazo_sini', 4)","('periodo_rechazo_sini', 5)","('periodo_rechazo_sini', 6)","('periodo_rechazo_sini', 7)","('periodo_rechazo_sini', 8)","('periodo_rechazo_sini', 9)","('periodo_rechazo_sini', 10)","('periodo_rechazo_sini', 11)","('periodo_rechazo_sini', 12)","('periodo_rechazo_sini', 13)"
0,1482.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1559.0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4655.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5231.0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,7870.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


** Cambio de nombre de columnas **

In [28]:
df_sini.columns

Index(['CIF_ID', '('NUM_SECU_POL', 1)', '('NUM_SECU_POL', 2)',
       '('NUM_SECU_POL', 3)', '('NUM_SECU_POL', 4)', '('NUM_SECU_POL', 5)',
       '('NUM_SECU_POL', 6)', '('NUM_SECU_POL', 7)', '('NUM_SECU_POL', 8)',
       '('NUM_SECU_POL', 9)', '('NUM_SECU_POL', 10)', '('NUM_SECU_POL', 11)',
       '('NUM_SECU_POL', 12)', '('NUM_SECU_POL', 13)',
       '('periodo_liquidacion_sini', 1)', '('periodo_liquidacion_sini', 2)',
       '('periodo_liquidacion_sini', 3)', '('periodo_liquidacion_sini', 4)',
       '('periodo_liquidacion_sini', 5)', '('periodo_liquidacion_sini', 6)',
       '('periodo_liquidacion_sini', 7)', '('periodo_liquidacion_sini', 8)',
       '('periodo_liquidacion_sini', 9)', '('periodo_liquidacion_sini', 10)',
       '('periodo_liquidacion_sini', 11)', '('periodo_liquidacion_sini', 12)',
       '('periodo_liquidacion_sini', 13)', '('periodo_rechazo_sini', 1)',
       '('periodo_rechazo_sini', 2)', '('periodo_rechazo_sini', 3)',
       '('periodo_rechazo_sini', 4)', '('per

In [29]:
df_sini = df_sini.rename(columns=lambda x: x.replace("(","").replace(")","").replace(", ","_").replace("'","").replace("NUM_SECU_POL","periodo_sini"))

In [30]:
df_sini.columns

Index(['CIF_ID', 'periodo_sini_1', 'periodo_sini_2', 'periodo_sini_3',
       'periodo_sini_4', 'periodo_sini_5', 'periodo_sini_6', 'periodo_sini_7',
       'periodo_sini_8', 'periodo_sini_9', 'periodo_sini_10',
       'periodo_sini_11', 'periodo_sini_12', 'periodo_sini_13',
       'periodo_liquidacion_sini_1', 'periodo_liquidacion_sini_2',
       'periodo_liquidacion_sini_3', 'periodo_liquidacion_sini_4',
       'periodo_liquidacion_sini_5', 'periodo_liquidacion_sini_6',
       'periodo_liquidacion_sini_7', 'periodo_liquidacion_sini_8',
       'periodo_liquidacion_sini_9', 'periodo_liquidacion_sini_10',
       'periodo_liquidacion_sini_11', 'periodo_liquidacion_sini_12',
       'periodo_liquidacion_sini_13', 'periodo_rechazo_sini_1',
       'periodo_rechazo_sini_2', 'periodo_rechazo_sini_3',
       'periodo_rechazo_sini_4', 'periodo_rechazo_sini_5',
       'periodo_rechazo_sini_6', 'periodo_rechazo_sini_7',
       'periodo_rechazo_sini_8', 'periodo_rechazo_sini_9',
       'periodo_rec

In [31]:
len(df_sini.columns)

40

In [32]:
df_sini.to_feather("siniestros_x_cif_id.feather")

In [33]:
df_sini.shape

(219405, 40)